In [1]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:35497")
client

<Client: 'tcp://127.0.0.1:35497' processes=9 threads=72, memory=0.98 TiB>

In [31]:
client.restart()

<Client: 'tcp://127.0.0.1:35497' processes=0 threads=0, memory=0 B>

## Time averaging with Thickness Weighted Averaging (TWA)

An alternative to averaging at a constant depth is to average following isopycnals. The averaging that results is referred to as TWA, and here we will evaluate the tracer equations in this frame. 

The model output was saved as snapshots at 3days for 16.5 years. Here we use that saved output to first transform to isopycnal coordinates, and then do a TWA in that. 



In [4]:
import xarray as xr
from xmitgcm import open_mdsdataset
from matplotlib import pyplot as plt
import os.path as op
import numpy as np
import xgcm 
%matplotlib inline

In [5]:
savedir = '/swot/SUM04/dbalwada/channel_model/05km_sponge/TWA/'

In [6]:
ds_dir = '/swot/SUM04/dbalwada/channel_model/05km_sponge/run_tracers_restored_3day_output/run_tracers_restored/'
ds = xr.open_zarr('/swot/SUM04/dbalwada/channel_model/05km_sponge/run_tracers_restored_3day_output/run_tracers_restored_zarr/')
# drop variables that have different axis, and not useful for TWA 
ds = ds.drop_vars(['Eta', 'PH', 'PHL','W']).reset_coords(drop=True)

In [7]:
ds

,Array,Chunk
Bytes,47.21 GiB,21.36 MiB
Shape,"(1980, 40, 400, 400)","(35, 1, 400, 400)"
Count,2281 Tasks,2280 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,47.21 GiB,21.36 MiB
Shape,"(1980, 40, 400, 400)","(35, 1, 400, 400)"
Count,2281 Tasks,2280 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


### Z -> b grid
First we cast to buoyancy surface. This involves moving all the variables from x,y,z,t grid to
x,y,b,t grid. Essentially go from a regular z grid to a regular b grid. 

In [8]:
grid = xgcm.Grid(ds, periodic='X')

In [9]:
ds['U'] = grid.interp(ds['U'], 'X')
ds['V'] = grid.interp(ds['V'], 'Y', boundary='extend')

In [10]:
Tcenter = np.linspace(0,8, 51)
dT = Tcenter[1] - Tcenter[0]
Touter = np.append(Tcenter - dT/2, Tcenter[-1] + dT/2)

In [11]:
# select a part of the data during initial calculations
#ds = ds.isel(time=slice(0, 800))

In [12]:
ds = ds.chunk({'Z':-1, 
               # 'time':100, 
               'XC':100, 'YC':100,
               'XG':100, 'YG':100, 'Zl':-1})

In [24]:
# cast available variables into isopycnal coordinates 
# ds_iso = xr.Dataset()

# for var_name in ds.keys():
#     print(var_name)
#     if var_name != 'T':
#         ds_iso[var_name] = grid.transform(ds[var_name], 'Z', Tcenter,
#                         target_data=ds['T'], method='linear'
#                                          )
        
# dsave = ds_iso.chunk({'T':10})
# dsave.encoding = {}
# dsave.to_zarr(op.join(savedir,'Tr-Mom-Remapped.zarr'), mode='w')
ds_iso = xr.open_zarr(op.join(savedir,'Tr-Mom-Remapped.zarr'))
ds_iso = ds_iso.assign_coords({'Tc':Touter})

In [14]:
# this is the variable \zeta 
# Z_grid = ds.Z*ds['T']/ds['T'] # convert Z array to a 3D field
Z_grid = ds.Z * xr.ones_like(ds['T'])
# ds_iso['zeta'] = grid.transform(Z_grid, 'Z', Tcenter, 
#                         target_data=ds['T'], method='linear'
#                                ).compute().chunk({'T':10,'YC':100,'XC':100,'time':35})
zeta = grid.transform(Z_grid, 'Z', Tcenter, 
                        target_data=ds['T'], method='linear'
                               )

In [25]:
grid_iso = xgcm.Grid(ds_iso, coords={'T':{'center':'T', 'outer':'Tc'}},
                periodic='X')

In [26]:
grid_iso

<xgcm.Grid>
T Axis (not periodic, boundary=None):
  * center   T --> outer
  * outer    Tc --> center

In [27]:
# # thickness
# # ds_iso['sigma'] = grid_iso.interp(grid_iso.diff(ds_iso['zeta'], 'T', boundary='extend')/ dT, 
# #                              'T', boundary='extend'
# #                                  ).compute().chunk({'T':10,'YC':100,'XC':100,'time':35})
# # thickness
# sigma = grid_iso.interp(grid_iso.diff(zeta, 'T', boundary='extend')/ dT, 
#                         'T', boundary='extend'
#                        )
# dsave = sigma.chunk({'T':10}).to_dataset(name='sigma')
# dsave.encoding = {}
# dsave.to_zarr(op.join(savedir,'sigmas.zarr'), mode='w')

sigma = xr.open_zarr(op.join(savedir,'sigmas.zarr')).sigma

In [28]:
# %%time
# # zeta_bar.load();
# # zeta_bar.to_netcdf('zeta_bar.nc')
# dsave = zeta_bar.chunk({'T':10,'XC':100,'YC':100}).to_dataset(name='zeta_bar')
# dsave.encoding = {}
# dsave.to_zarr('zeta_temp.zarr', mode='w')
# dsave.close()

zeta_bar = xr.open_zarr('zeta_temp.zarr').zeta_bar

In [29]:
# %%time
# # sigma_bar.load();
# # sigma_bar.to_netcdf('sigma_bar.nc')
# dsave = sigma_bar.chunk({'T':10,'XC':100,'YC':100}).to_dataset(name='sigma_bar')
# dsave.encoding = {}
# dsave.to_zarr('sigma_bar_temp.zarr', mode='w')
# dsave.close()

sigma_bar = xr.open_zarr('sigma_bar_temp.zarr').sigma_bar

In [20]:
# cast into averages  
ds_hat = xr.Dataset()

# sigma_bar = ds_iso['sigma'].mean('time').compute()
# zeta_bar = ds_iso['zeta'].mean('time').compute()


for var_name in ds_iso.keys():
    
    if (var_name != 'zeta' and var_name != 'sigma'):
        print(var_name)
        # ds_hat[var_name] = (ds_iso[var_name] * ds_iso['sigma']).mean('time')/sigma_bar
        ds_hat[var_name] = (ds_iso[var_name] * sigma
                           ).mean('time').compute().chunk({'T':10,'YC':100,'XC':100})/sigma_bar

PTRACER01
PTRACER02
PTRACER03
PTRACER04
PTRACER05
PTRACER06
PTRACER07
PTRACER08
PTRACER09
PTRACER10
PTRACER11
PTRACER12
PTRACER13
PTRACER14
PTRACER15
PTRACER16
PTRACER17
PTRACER18
PTRACER19
PTRACER20
U
V


In [17]:
zeta_bar.nbytes/1e9

0.12928

In [21]:
# %%time
# # ds_hat.load();
# # ds_hat.to_netcdf('ds_hat.nc')
# dsave = ds_hat.chunk({'T':10})
# dsave.encoding = {}
# dsave.to_zarr('ds_hat.zarr', mode='w')
# dsave.close()

ds_hat = xr.open_dataset('ds_hat.zarr')

CPU times: user 2.23 s, sys: 874 ms, total: 3.11 s
Wall time: 8.81 s


In [ ]:
1+1

In [32]:
# eddy corrs 

ds_twa_eddy_u = xr.Dataset()

# upp = ds_iso['U'] - ds_hat['U']
# vpp = ds_iso['V'] - ds_hat['V']

for var_name in ds_hat.keys():
    if (var_name != 'U' and var_name != 'V'):
        print(var_name)
        
        # ds_twa_eddy_u['U_'+var_name] = ((ds_iso[var_name] - ds_hat[var_name])*upp*ds_iso['sigma']).mean('time')/sigma_bar
        # ds_twa_eddy_v['V_'+var_name] = ((ds_iso[var_name] - ds_hat[var_name])*vpp*ds_iso['sigma']).mean('time')/sigma_bar
        ds_twa_eddy_u['U_'+var_name] = ((sigma*ds_iso[var_name]*ds_iso['U']).mean('time') 
                                        - ds_hat[var_name]*ds_hat['U']
                                       ).compute().chunk({'T':10,'YC':100,'XC':100})/sigma_bar

PTRACER01
PTRACER02
PTRACER03
PTRACER04
PTRACER05
PTRACER06
PTRACER07
PTRACER08
PTRACER09
PTRACER10
PTRACER11
PTRACER12
PTRACER13
PTRACER14
PTRACER15
PTRACER16
PTRACER17
PTRACER18
PTRACER19
PTRACER20


In [33]:
%%time
# ds_twa_eddy_u.load();
# ds_twa_eddy_u.to_netcdf('ds_twa_eddy_u.nc')
dsave = ds_twa_eddy_u.chunk({'T':10})
dsave.encoding = {}
dsave.to_zarr(op.join(savedir,'ds_twa_eddy_u.zarr'), mode='w')
dsave.close()

CPU times: user 1.71 s, sys: 895 ms, total: 2.6 s
Wall time: 6.62 s


In [34]:
ds_twa_eddy_v = xr.Dataset()

for var_name in ds_hat.keys():
    if (var_name != 'U' and var_name != 'V'):
        print(var_name)
        
        ds_twa_eddy_v['V_'+var_name] = ((sigma*ds_iso[var_name]*ds_iso['V']).mean('time') 
                                        - ds_hat[var_name]*ds_hat['V']
                                       ).compute().chunk({'T':10,'YC':100,'XC':100})/sigma_bar

PTRACER01
PTRACER02
PTRACER03
PTRACER04
PTRACER05
PTRACER06
PTRACER07
PTRACER08
PTRACER09
PTRACER10
PTRACER11
PTRACER12
PTRACER13
PTRACER14
PTRACER15
PTRACER16
PTRACER17
PTRACER18
PTRACER19
PTRACER20


In [35]:
%%time
# ds_twa_eddy_v.load();
# ds_twa_eddy_v.to_netcdf('ds_twa_eddy_v.nc')
dsave = ds_twa_eddy_v.chunk({'T':10})
dsave.encoding = {}
dsave.to_zarr(op.join(savedir,'ds_twa_eddy_v.zarr'), mode='w')
dsave.close()

CPU times: user 2.56 s, sys: 998 ms, total: 3.56 s
Wall time: 6.51 s
